In [1]:
import sys
import os
# Add the parent directory to the Python path

# Change the current working directory to /app
os.chdir('/app')
from dotenv import load_dotenv
load_dotenv(dotenv_path='config/.env')
database_path = os.getenv('DATABASE_PATH')
from utils.db.fetch import fetch_entries
print(f'DATABASE_PATH: {database_path}')
market_name = 'crypto_binance'
timeframe='1m'
pair='BTC'

DATABASE_PATH: database/db/ohlcv_data.db


In [2]:
from finstore.finstore import Finstore
finstore = Finstore(market_name=market_name, timeframe=timeframe, enable_append=True)

Len items in 'database/finstore/market_name=crypto_binance/timeframe=1m': 10


In [3]:
df = finstore.stream.fetch_trade_data(symbol='ETHBTC')
df.tail(10)

,e,E,s,k
0,kline,1734181168001,ETHBTC,"{'B': '0', 'L': 481022149, 'Q': '0.24190933', ..."
1,kline,1734181176000,ETHBTC,"{'B': '0', 'L': 481022150, 'Q': '0.24190933', ..."
2,kline,1734181188001,ETHBTC,"{'B': '0', 'L': 481022151, 'Q': '0.27150276', ..."
3,kline,1734181196001,ETHBTC,"{'B': '0', 'L': 481022153, 'Q': '0.27640253', ..."
4,kline,1734181200008,ETHBTC,"{'B': '0', 'L': 481022155, 'Q': '0.29663940', ..."
5,kline,1734181200101,ETHBTC,"{'B': '0', 'L': 481022155, 'Q': '0.29663940', ..."


In [9]:
import asyncio
import websockets
import json
from data.fetch.crypto_binance import fetch_symbol_list_binance

# Stop signal
stop_signal = False

# Function to handle incoming messages
async def handle_message(pair, message):
    print(f"Received for {pair}: {message}")
    finstore.stream.save_trade_data(pair, message)

def fetch_symbol_list_binance_spoof(type='spot', suffix='BTC'):
    return ["ETHBTC", "ADABTC", "XRPBTC"] 

# WebSocket handler with a stop mechanism
async def fetch_live_data():
    global stop_signal
    base_url = "wss://stream.binance.com:9443/ws"
    symbols = fetch_symbol_list_binance(type='spot', suffix='BTC')
    symbols = [symbol.replace('/', '') for symbol in symbols]
    streams = [f"{symbol.lower()}@trade" for symbol in symbols]
    url = f"{base_url}/{'/'.join(streams)}"
    
    async with websockets.connect(url, ping_interval=20, ping_timeout=10) as websocket:
        print("Connected to Binance WebSocket")
        while not stop_signal:
            try:
                data = await websocket.recv()
                message = json.loads(data)
                symbol = message.get("s", "Unknown Pair")
                await handle_message(symbol, message)
            except websockets.exceptions.ConnectionClosed as e:
                print(f"WebSocket closed: {e}")
                break
            except Exception as e:
                print(f"Error: {e}")
                break
        print("Exiting WebSocket connection...")

# Adjusted entry point for environments with a running event loop
async def main():
    await fetch_live_data()


# Function to stop the WebSocket fetcher
def stop_fetching():
    global stop_signal
    stop_signal = True
    print("Stop signal sent.")


In [ ]:
fetch_task = asyncio.create_task(main())

In [5]:
stop_fetching()

Stop signal sent.


For subscribing to minutely streams for every symbol

In [3]:
import asyncio
import websockets
import json
from data.fetch.crypto_binance import fetch_symbol_list_binance

# Stop signal
stop_signal = False

# Function to handle incoming messages
async def handle_message(pair, message):
    print(f"Received for {pair}: {message.get('k', {})}")
    finstore.stream.save_trade_data(pair, message, preset='binance_kline')

def fetch_symbol_list_binance_spoof(type='spot', suffix='BTC'):
    return ["ETHBTC", "ADABTC", "XRPBTC"] 

# WebSocket handler with a stop mechanism
async def fetch_live_data():
    global stop_signal
    base_url = "wss://stream.binance.com:9443/ws"
    symbols = fetch_symbol_list_binance(type='spot', suffix='BTC')
    symbols = [symbol.replace('/', '') for symbol in symbols]
    interval = "1m"  # Set the desired interval here
    streams = [f"{symbol.lower()}@kline_{interval}" for symbol in symbols]  # Updated to kline stream
    url = f"{base_url}/{'/'.join(streams)}"
    
    while not stop_signal:
        try:
            async with websockets.connect(url, ping_interval=180, ping_timeout=600) as websocket:
                print("Connected to Binance WebSocket")
                while not stop_signal:
                    try:
                        data = await websocket.recv()
                        message = json.loads(data)
                        symbol = message.get("s", "Unknown Pair")
                        await handle_message(symbol, message)
                    except websockets.exceptions.ConnectionClosed as e:
                        print(f"WebSocket closed: {e}")
                        break
                    except Exception as e:
                        print(f"Error: {e}")
                        break
        except Exception as e:
            print(f"Connection error: {e}")
        
        if not stop_signal:
            print("Reconnecting in 5 seconds...")
            await asyncio.sleep(5)
    
        print("Exiting WebSocket connection...")

# Adjusted entry point for environments with a running event loop
async def main():
    await fetch_live_data()


# Function to stop the WebSocket fetcher
def stop_fetching():
    global stop_signal
    stop_signal = True
    print("Stop signal sent.")
